In [ ]:
!pip3 install pandas --quiet

In [6]:
import sqlite3
import pandas as pd
import os

# Connect to (or create) a database
conn = sqlite3.connect('rentals.db')

In [85]:
file_paths = [os.path.join('data', i) for i in os.listdir('data')]

In [92]:
for i,v in enumerate(file_paths):
    file_name =  file_paths[i].split("/")[1].split(".")[0]
    globals()[file_paths[i].split("/")[1].split(".")[0]] = pd.read_csv(v)
    globals()[file_paths[i].split("/")[1].split(".")[0]].to_sql(file_name, conn,
                                    if_exists='replace', index=False)
    print(f"{file_paths[i].split("/")[1].split(".")[0]} table created succesfully")

membership_type table created succesfully
customer table created succesfully
membership table created succesfully
bike table created succesfully
rental table created succesfully


Emily would like to know how many bikes the shop owns by category. Can 
you get this for her?  Display the category name and the number of bikes the shop owns in 
each category (call this column  `number_of_bikes` ). Show only the categories 
where the number of bikes is greater than 2 

In [25]:
query = '''
select category, count(*) as number_of_bikes from bike
group by category
'''
temp1 = pd.read_sql(query, conn)
temp1

,category,number_of_bikes
0,electric,2
1,hybrid,2
2,mountain bike,3
3,road bike,3


Emily needs a list of customer names with the total number of 
memberships purchased by each. For each customer, display the customer's name and the count of  memberships purchased (call this column  membership_count ). Sort the 
results by  membership_count , starting with the customer who has purchased 
the highest number of memberships. Keep in mind that some customers may not have purchased any memberships yet. In such a situation, display  0  for the  membership_count .

In [49]:
query = '''
select t1.name, count(t2.id) as membership_count from customer t1
    left join membership t2 on t1.id = t2.customer_id
    group by 1
    order by 2 desc, 1 asc
'''
temp2 = pd.read_sql(query, conn)
temp2

,name,membership_count
0,Alice Smith,3
1,Bob Johnson,3
2,Eva Brown,2
3,John Doe,2
4,Michael Lee,2
5,Daniel Miller,0
6,David Wilson,0
7,Emily Davis,0
8,Olivia Taylor,0
9,Sarah White,0


In [48]:
query = '''
with cte as (
select distinct t1.id, t1.category, t1.price_per_hour as old_price_per_hour,
t1.price_per_day as old_price_per_day
 from bike t1 left join rental t2 on t1.id =  t2.bike_id)

select *, 
    case when category = 'electric' then  old_price_per_hour * 0.9
            when category = 'mountain_bike' then  old_price_per_hour * 0.8
            else old_price_per_hour * 0.5 end as new_price_per_hour,

    case when category = 'electric' then  old_price_per_day * 0.8
            when category = 'mountain_bike' then  old_price_per_day * 0.5
            else old_price_per_day * 0.5 end as new_price_per_day
from cte
'''
temp3 = pd.read_sql(query, conn)
temp3

,id,category,old_price_per_hour,old_price_per_day,new_price_per_hour,new_price_per_day
0,1,mountain bike,10,50,5.0,25.0
1,2,road bike,12,60,6.0,30.0
2,3,hybrid,8,40,4.0,20.0
3,4,electric,15,75,13.5,60.0
4,5,mountain bike,10,50,5.0,25.0
5,6,road bike,12,60,6.0,30.0
6,7,hybrid,8,40,4.0,20.0
7,8,electric,15,75,13.5,60.0
8,9,mountain bike,10,50,5.0,25.0
9,10,road bike,12,60,6.0,30.0


Emily is looking for counts of the rented bikes and of the available bikes in 
each category. Display the number of available bikes (call this column 
available_bikes_count ) and the number of rented bikes (call this column 
rented_bikes_count ) by bike category.

In [53]:
query = '''
select sum(case when t1.status = 'available' then 1 else 0 end) as available_bikes_count,
       sum(case when t1.status = 'rented' then 1 else 0 end) as available_bikes_count  from bike t1 
join rental t2 on t1.id =  t2.bike_id
'''
pd.read_sql(query, conn)

,available_bikes_count,available_bikes_count
0,22,7


Emily is preparing a sales report. She needs to know the total revenue 
from rentals by month, the total by year, and the all-time across all the 
years. 

Display the total revenue from rentals for each month, the total for each 
year, and the total across all the years. Do not take memberships into 
account. 

There should be 3 columns:  year ,  month , and  revenue .
Sort the results chronologically. Display the year total after all the month 
totals for the corresponding year. Show the all-time total as the last row.
The resulting table looks something like this:


| Year | Month | Revenue |
|------|-------|---------|
| 2022 | 11    | 200.00  |
| 2022 | 12    | 150.00  |
| 2022 | null  | 350.00  |
| 2023 | 1     | 110.00  |
| ...  | ...   | ...     |
| 2023 | 10    | 335.00  |
| 2023 | null  | 1370.00 |
| null | null  | 1720.00 |

In [61]:
query = '''
WITH cte AS (
    SELECT strftime("%Y", start_timestamp) AS year,
           strftime("%m", start_timestamp) AS month,
           total_paid
    FROM rental
)
SELECT year, month, SUM(total_paid) AS Revenue
FROM cte
GROUP BY year, month

UNION ALL

SELECT 'Total', 'Total', SUM(total_paid) AS Revenue
FROM cte;
'''
pd.read_sql(query, conn)

,year,month,Revenue
0,2022,11,200
1,2022,12,150
2,2023,01,110
3,2023,02,40
4,2023,03,110
5,2023,04,90
6,2023,05,120
7,2023,06,115
8,2023,07,150
9,2023,08,125


I am using `SQLite3` but this can be done using ROLLUP function in another RDBMS

```sql
WITH cte AS (
    SELECT year(start_timestamp) AS year,
           month(start_timestamp) AS month,
           total_paid
    FROM rental
)
SELECT year, month, SUM(total_paid) AS Revenue
FROM cte
GROUP BY ROLLUP (year, month)
```

Emily has asked you to get the total revenue from memberships for each 
combination of year, month, and membership type.
Display the year, the month, the name of the membership type (call this 
column  membership_type_name ), and the total revenue (call this column 
total_revenue ) for every combination of year, month, and membership type. 
Sort the results by year, month, and name of membership type.

In [75]:
query = '''
WITH cte as (
SELECT strftime("%Y", start_date) AS year, strftime("%m", start_date) as month,
     membership_type_id, SUM(total_paid) AS total_revenue
FROM membership
GROUP BY 1,2, 3)

select t1.year,t1.month, t2.name as membership_type_name, t1.total_revenue from cte t1 
     join membership_type t2 on t1.membership_type_id =  t2.id order by 1,2,3
'''
pd.read_sql(query, conn)

,year,month,membership_type_name,total_revenue
0,2023,08,Basic Annual,500
1,2023,08,Basic Monthly,100
2,2023,08,Premium Monthly,200
3,2023,09,Basic Annual,500
4,2023,09,Basic Monthly,100
5,2023,09,Premium Monthly,200
6,2023,10,Basic Annual,500
7,2023,10,Basic Monthly,100
8,2023,10,Premium Monthly,200
9,2023,11,Basic Annual,500


Next, Emily would like data about memberships purchased in 2023, with 
subtotals and grand totals for all the different combinations of membership 
types and months.
Display the total revenue from memberships purchased in 2023 for each 
combination of month and membership type. Generate subtotals and 
grand totals for all possible combinations.  There should be 3 columns: 
membership_type_name ,  month , and  total_revenue .
Sort the results by membership type name alphabetically and then 
chronologically by month.


Now it's time for the final task.
Emily wants to segment customers based on the number of rentals and 
see the count of customers in each segment. Use your SQL skills to get 
this!
Categorize customers based on their rental history as follows:
Customers who have had more than 10 rentals are categorized as  'more 
than 10' .
Customers who have had 5 to 10 rentals (inclusive) are categorized as 
'between 5 and 10' .
Customers who have had fewer than 5 rentals should be categorized as 
'fewer than 5' .
Calculate the number of customers in each category. Display two columns: 
rental_count_category  (the rental count category) and  customer_count  (the 
number of customers in each category)